# Fine-Tune BLOOM with Reinforcement Learning (PPO) and PEFT to Generate Less-Toxic Summaries for Azerbaijani Langueges

In this notebook, you will fine-tune a BLOOM model to generate less toxic content with Meta AI's hate speech reward model. The reward model is a binary classifier that predicts either "not hate" or "hate" for the given text. You will use Proximal Policy Optimization (PPO) to fine-tune and reduce the model's toxicity.